In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix


In [2]:
DATASET_DIR = "data_split"   # folder containing train/val/test

TRAIN_DIR = os.path.join(DATASET_DIR, "train")
VAL_DIR   = os.path.join(DATASET_DIR, "validation")
TEST_DIR  = os.path.join(DATASET_DIR, "test")

## Hyperparameters

In [3]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 100

## Data Generators

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [8]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

validation_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)


Found 483 images belonging to 53 classes.
Found 103 images belonging to 53 classes.
Found 110 images belonging to 53 classes.


In [9]:
base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

## FREEZE FIRST 20 LAYERS

In [10]:
for layer in base_model.layers[:120]:
    layer.trainable = False

for layer in base_model.layers[120:]:
    layer.trainable = True


## Custom Classification Head

In [11]:
x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = Dropout(0.6)(x)

output = Dense(train_generator.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

In [12]:
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [13]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_2[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 24,125,877 (92.03 MB)

 Trainable params: 17,753,653 (67.72 MB)

 Non-trainable params: 6,372,224 (24.31 MB)

## Train model

In [14]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks=[early_stop]
)

Epoch 1/100


2026-01-08 12:23:52.039877: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


16/16 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.0373 - loss: 4.7791 - val_accuracy: 0.0000e+00 - val_loss: 5.0167
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.0518 - loss: 4.3450 - val_accuracy: 0.0000e+00 - val_loss: 4.8982
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.1201 - loss: 4.0380 - val_accuracy: 0.0000e+00 - val_loss: 4.7983
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.1387 - loss: 3.9872 - val_accuracy: 0.0000e+00 - val_loss: 4.7063
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.1822 - loss: 3.8282 - val_accuracy: 0.0000e+00 - val_loss: 4.6428
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.2008 - loss: 3.6633 - val_accuracy: 0.0000e+00 - val_loss: 4.5939
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.2464 - loss: 3.5266 - val_accuracy: 0.0000e+00 - val_loss: 4.5459
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.2464 - loss: 3.4461 - val_accu

In [15]:
model.evaluate(test_generator)

4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.8273 - loss: 1.1925


[1.1924829483032227, 0.8272727131843567]

In [16]:
model.save("attendence_transfer.keras") # Saving the model as "attendence.keras"

In [17]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import os
import pathlib


model = load_model("attendence.keras")


data_dir = pathlib.Path("./data_split/train")

class_names = np.array([
    item.name
    for item in data_dir.glob("*")
    if item.is_dir() and not item.name.startswith(".")
])

print(class_names)


# Face detector
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# Prediction function
def predict_face(face):
    # resizing the image (224, 224) to fit into the model and normalize it
    face_resized = cv2.resize(face, (224, 224))  # resizing the image (224, 224) to fit into the model
    face_resized = face_resized / 255.0
    face_expanded = np.expand_dims(face_resized, axis=0)

    preds = model.predict(face_expanded)[0]

    # Highest scoring class
    class_id = np.argmax(preds)
    confidence = preds[class_id] * 100  # Convert to %

    return class_names[class_id], confidence


# Webcam loop
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]

        s_id, confidence = predict_face(face_roi)

        # if confidence is greater or equal to 95% then it will show the id other wise "Unknown"
        if confidence >= 95:
            text = f"{s_id} ({confidence:.2f}%)"
            color = (0, 255, 0)
        else:
            text = "Unknown"
            color = (0, 0, 255)

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, text, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Attendance System", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): # for quitting press "q"
        break

cap.release()
cv2.destroyAllWindows()


['22-47884-2' '22-49862-3' '22-48569-3' '22-48133-2 ' '22-49843-3'
 '22-49037-3' '22-46258-1' '22-46139-1' '22-49451-3' '23-50346-1'
 '23-51308-1' '22-48205-2' '22-46590-1' '22-49355-3' '22-49370-3'
 '23-51127-1' '22-48005-2' '22-49783-3' '23-50277-1' '22-46342-1'
 '21-45902-3' '22-48682-3' '22-49824-3' '22-48725-3' '22-49575-3'
 '23-50066-1' '22-46138-1' '22-48582-3' '22-48915-3' '22-47892-2'
 '22-47898-2' '22-48841-3' '22-46536-1' '22-48021-2' '22-48023-2_'
 '22-49196-3' '22-49331-3' '22-48666-3' '22-49791-3' '22-48091-2'
 '22-49068-3' '22-47968-2' '22-49538-3' '22-48833-3' '23-50254-1'
 '22-47542-2' '22-49507-3' '22-49852-3' '22-46983-1' '22-47813-2'
 '22-49643-3' '22-49338-3' '22-46887-1']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12m